In [26]:
from operator import itemgetter

In [97]:
import catboost as cb
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.base import clone
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [3]:
from basedir import SAMPLE
from info import id_cols
from utils import to_feather, from_feather

In [4]:
x_trn, y_trn, x_tst = from_feather('x_trn', 'y_trn', 'x_tst')

In [6]:
x_trn.head()

,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003386,0.33995,1.5064,-9.4128
2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008973,0.42684,1.0993,-10.0960
4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [5]:
y_trn.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [8]:
x_tst.head()

,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0,0,-0.025773,-0.98864,-0.14801,0.003350,-0.006524,-0.001071,-0.027390,0.10043,4.2061,-5.5439
1,0,1,-0.025683,-0.98862,-0.14816,0.003439,-0.113960,0.083987,-0.060590,-0.70889,3.9905,-8.0273
2,0,2,-0.025617,-0.98861,-0.14826,0.003571,-0.080518,0.114860,-0.037177,1.45710,2.2828,-11.2990
3,0,3,-0.025566,-0.98862,-0.14817,0.003609,0.070067,0.033820,-0.035904,0.71096,1.8582,-12.2270
4,0,4,-0.025548,-0.98866,-0.14792,0.003477,0.152050,-0.029016,-0.015314,3.39960,2.7881,-10.4100


In [ ]:
def dropcols(df, cols, missing_ok=True):
    cols = df.columns[df.columns.isin(cols)]

In [81]:
def kmeans_cluster(ser_id, data, k=5):
    ser = data[data.series_id == ser_id].copy()
    del ser['series_id'], ser['measurement_number']
    kmeans = KMeans(n_clusters=k).fit(ser)
    vec = kmeans.cluster_centers_.flatten()
    return ser_id, vec

In [98]:
def cluster(data, func, **params):
    with joblib.Parallel(n_jobs=12, backend='loky') as parallel:
        results = parallel(
            joblib.delayed(func)(ser_id, group, **params)
            for ser_id, group in data.groupby('series_id'))
    _, vectors = zip(*sorted(results, key=itemgetter(0)))
    return np.row_stack(vectors)

In [100]:
scaler = StandardScaler()
x_trn.drop(columns=id_cols)

/home/ck/anaconda3/envs/fastai/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint32, float32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ck/anaconda3/envs/fastai/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint32, float32 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/ck/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint32, float32 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
trn_orient = cluster(x_trn, kmeans_cluster)

In [83]:
_, vectors = zip(*sorted(results, key=itemgetter(0)))

In [84]:
x_trn_new = np.row_stack(vectors)

In [85]:
with joblib.Parallel(n_jobs=12, backend='loky') as parallel:
    results = parallel(
        joblib.delayed(kmeans_cluster)(ser_id, data) 
        for ser_id, data in x_tst.groupby('series_id'))

In [86]:
_, vectors = zip(*sorted(results, key=itemgetter(0)))

In [87]:
x_tst_new = np.row_stack(vectors)

In [88]:
enc = LabelEncoder()
y_enc = enc.fit_transform(y_trn['surface'])

In [89]:
model = RandomForestClassifier(n_estimators=1000, n_jobs=-1)

In [90]:
model.fit(x_trn_new, y_enc)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [91]:
preds = model.predict(x_tst_new)

In [92]:
submit = pd.read_csv(SAMPLE)
submit['surface'] = enc.classes_[preds]
submit.to_csv('submit.csv', index=None)

In [93]:
!kaggle c submit career-con-2019 -f 'submit.csv' -m "test (less)"

100%|██████████████████████████████████████| 53.3k/53.3k [00:00<00:00, 63.9kB/s]
Successfully submitted to CareerCon 2019 - Help Navigate Robots 